In [21]:
pip install praw


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import praw
import sys
import re, numpy as np, pandas as pd
from pprint import pprint
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models, similarities
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import requests 
import time 
import bs4
from bs4 import BeautifulSoup
import random


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
#https://www.reddit.com/prefs/apps

reddit = praw.Reddit(client_id = 'NfUTs733BtFmTbLJ7Q_1fA',client_secret = 'Cj_PSEMGJwJNAV4hvJliy-SE6qrOiQ', user_agent = 'will')
submission = reddit.submission(url="https://www.reddit.com/r/wallstreetbets/comments/x9rf74/daily_discussion_thread_for_september_09_2022/")



In [25]:
text = []
from praw.models import MoreComments
for top_level_comment in submission.comments:
    if isinstance(top_level_comment, MoreComments):
        continue
    text.append(top_level_comment.body)
print(len(text))
df = pd.DataFrame()
df['text'] = text

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



471


In [26]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', str(sent))  
        sent = re.sub('\s+', ' ', str(sent))  
        sent = re.sub("\'", "", str(sent))  
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  
        
def cleaner(df):
  data = df.text.values.tolist()
  data_words = list(sent_to_words(data))
  return data_words, data

def topicMaster(target,num_topics, savedAs):

  data_words, data = cleaner(target)
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
      texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
      texts = [bigram_mod[doc] for doc in texts]
      texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
      texts_out = []
      nlp = spacy.load('en_core_web_sm')
      for sent in texts:
          doc = nlp(" ".join(sent)) 
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
      return texts_out
  data_ready = process_words(data_words)
  id2word = corpora.Dictionary(data_ready)

  corpus = [id2word.doc2bow(text) for text in data_ready]
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=10,
                                            passes=10,
                                            alpha='symmetric',
                                            iterations=100,
                                            per_word_topics=True)

  pprint(lda_model.print_topics())

  def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
      sent_topics_df = pd.DataFrame()
      for i, row_list in enumerate(ldamodel[corpus]):
          row = row_list[0] if ldamodel.per_word_topics else row_list            
          row = sorted(row, key=lambda x: (x[1]), reverse=True)
          for j, (topic_num, prop_topic) in enumerate(row):
              if j == 0: 
                  wp = ldamodel.show_topic(topic_num)
                  topic_keywords = ", ".join([word for word, prop in wp])
                  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
              else:
                  break
      sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
      sent_topics_df=sent_topics_df[sent_topics_df['Dominant_Topic']>=.5]
      contents = pd.Series(texts)
      sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
      return(sent_topics_df)

  df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)
  df_dominant_topic = df_topic_sents_keywords.reset_index()
  df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

  pd.options.display.max_colwidth = 100
  sent_topics_sorteddf_mallet = pd.DataFrame()
  sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
  for i, grp in sent_topics_outdf_grpd:
      sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                              grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                              axis=0)
  sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)
  sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

  import pyLDAvis
  import pyLDAvis.gensim_models
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
  p = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
  pyLDAvis.save_html(p, savedAs)
  return vis

topicMaster(df,5, 'new_stock_topics.html')

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[(0,
  '0.092*"dead" + 0.079*"spy" + 0.036*"dump" + 0.032*"shit" + 0.024*"already" '
  '+ 0.023*"rip" + 0.019*"time" + 0.019*"talk" + 0.014*"happen" + '
  '0.014*"play"'),
 (1,
  '0.072*"img_emote_th" + 0.069*"buy" + 0.049*"market" + 0.047*"call" + '
  '0.031*"pump" + 0.031*"stop" + 0.015*"die" + 0.013*"cpi" + 0.013*"recession" '
  '+ 0.010*"long"'),
 (2,
  '0.088*"ber" + 0.075*"inside" + 0.075*"outside" + 0.043*"today" + '
  '0.026*"back" + 0.019*"week" + 0.018*"money" + 0.012*"hold" + 0.012*"lose" + '
  '0.011*"next"'),
 (3,
  '0.034*"close" + 0.030*"lol" + 0.027*"bear" + 0.023*"weekend" + 0.018*"love" '
  '+ 0.015*"fuk" + 0.014*"girl" + 0.014*"bullish" + 0.013*"meme" + '
  '0.013*"work"'),
 (4,
  '0.068*"put" + 0.036*"bull" + 0.032*"fuck" + 0.022*"sell" + 0.022*"open" + '
  '0.021*"day" + 0.019*"rally" + 0.017*"let" + 0.015*"ass" + 0.015*"double"')]


/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.124963 -0.158938       1        1  24.012501
1     -0.096808  0.126477       2        1  22.245255
2      0.195628 -0.108463       3        1  20.677623
3     -0.075086  0.003771       4        1  16.591186
0      0.101230  0.137153       5        1  16.473436, topic_info=              Term       Freq      Total Category  logprob  loglift
329            ber  48.000000  48.000000  Default  30.0000  30.0000
225           dead  40.000000  40.000000  Default  29.0000  29.0000
47    img_emote_th  42.000000  42.000000  Default  28.0000  28.0000
40             spy  34.000000  34.000000  Default  27.0000  27.0000
10             put  43.000000  43.000000  Default  26.0000  26.0000
...            ...        ...        ...      ...      ...      ...
807        balance   1.922477   2.734020   Topic5  -5.4143   1.4513
1002         alive   2.429169   4.119158   Topic5  -5.1803   1.2753
985         record   3.663164   9.643550   Topic5  -4.7696   0.8355
1010        inside   3.114926  44.558870   Topic5  -4.9317  -0.8572
1011       outside   3.114926  44.558870   Topic5  -4.9317  -0.8572

[199 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
65        5  0.955439    account
551       5  0.610784     actual
965       4  0.642633     adjust
1002      3  0.242768      alive
1002      5  0.485536      alive
...     ...       ...        ...
16        4  0.971124       work
681       2  0.845297      worth
7         4  0.741962      wrong
95        4  0.758862       year
118       2  0.953498  yesterday

[189 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 4, 1])

In [8]:
for posts in submission:
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
print(posts)



TypeError: ignored